In [21]:
from openpyxl import Workbook
import pandas as pd
import re
import xlwings 

In [22]:
dat = {
    'cod_alumno':'cod',
    'des_doc_identidad':'doc_type',
    'did_alumno':'doc_num',
    'ape_paterno':'paternal_name',
    'ape_materno':'maternal_name',
    'nom_alumno':'name',
    'tel_alu_movil':'cel_num',
    'tel_alumno':'phone_num',
    'coe_alumno':'inst_email',
    'coe_alu_personal':'per_email',
    'anio_ingreso':'in_year',
    'periodo_egreso':'grad_per',
    'escuela':'school'
}
col = list(dat.values())
archivos =['./EGRESADOS/BASE DE DATOS AUDITORIA EGRESADOS.xlsx','./EGRESADOS/BASE DE DATOS CONTABILIDAD EGRESADOS.xlsx','./EGRESADOS/BASE DE DATOS EGRESADOS GESTIÓN PÚBLICA.xlsx']
dic_school={'auditoría':'1','contabilidad':'2','gestión pública':'3'}
dic_doctype={'Doc. Nacional de Identidad':'1','Pasaporte':'2','Partida Nacimiento':'3','Otro':'4','Libreta Militar':'5','Boleta Militar':'5','Carnet de Extranjeria':'6','Carnet Identidad':'7'}

In [23]:
data = [pd.read_excel(archivo,sheet_name=1,usecols=dat.keys(),dtype="string") for archivo in archivos]

In [24]:
def document(val):
    if len(val)!=8 and len(val)!=10:
        val = None
    return val

In [25]:
db = pd.concat(data).rename(columns=dat).drop_duplicates().reset_index(drop=True)
db = db[db.doc_num.notnull()]
db.doc_num = db.doc_num.apply(document)
db = db[db.doc_num.notnull()]
db.fillna('sin_registro',inplace=True)

In [26]:
db = db.drop(index= [10817,1212,1388,11236,11271,6145,1212],axis=0)
db = db.reset_index(drop=True)

In [27]:
for val in db.doc_num:
    if len(val)==9:
        print(val)

In [28]:
def phone(val):
    if len(val)<7:
        val = 'sin_registro'
    if val[3]=='-' or val[3]==' ':
        val=val[:3]+val[4:]
    if re.search('^0',val):
        val=val[2:]
    if not re.search('^\d+',val) or re.search('^0',val):
        val='sin_registro'
    if len(val)!=7 and val != 'sin_registro':
        val = 'sin_registro'
    return val

def movil(val):
    if len(val)<9:
        val = 'sin_registro'
    if val[3]=='-':
        val=val[:3]+val[4:]
    if val[3]==' ' and val[7]==' ':
        val=val[:3]+val[4:7]+val[8:]
    if val[5]=='-' or val[5]==' ':
        val = val[0:5]+val[6:]
    if not re.search('^\d+',val):
        val = 'sin registro'
    if len(val)>9 and re.search('^51',val):
        val = val[2:]
    if len(val)>9:
        val = val[0:9]
    if not re.search('^9',val):
        val ='sin registro'
    if len(val)!=9:
        val = 'sin_registro'
    return val
    
def income(val):
    if len(val)!=4:
        val = 'sin_registro'
    return val

def graduated(val):
    if not re.search('^\d+',val) or re.search('^0',val):
        val = 'sin_registro'
    if val!='sin_registro' and val[4]!='-':
        val = val[:4] + '-' + val[4]
    return val

In [29]:
db.phone_num=db.phone_num.apply(phone)
db.cel_num=db.cel_num.apply(movil)
db.in_year=db.in_year.apply(income)
db.grad_per=db.grad_per.apply(graduated)

In [30]:
db = db[col]
db.columns = db.columns.str.upper()
db = db.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})
db.to_excel('EGRESA2.xlsx', sheet_name = 'EGRESADOS', index=False)

In [31]:
stu = ['COD','PATERNAL_NAME','MATERNAL_NAME','NAME','DOC_NUM','DOC_TYPE','SCHOOL']
tempo = ['PHONE_NUM','CEL_NUM','INST_EMAIL','PER_EMAIL','IN_YEAR','GRAD_PER']
doc = ['DOC_TYPE']
sch = ['SCHOOL']

In [32]:
student = pd.read_excel('./EGRESA2.xlsx',usecols=stu,dtype='string')
temporal = pd.read_excel('./EGRESA2.xlsx',usecols=tempo,dtype='string')
doc_type = pd.read_excel('./EGRESA2.xlsx',usecols=doc,dtype='string')
school = pd.read_excel('./EGRESA2.xlsx',usecols=sch,dtype='string')

In [33]:
school = school.drop_duplicates().reset_index(drop=True)
doc_type = doc_type.drop_duplicates().reset_index(drop=True)

In [34]:
temporal['CEL_NUM']= temporal['CEL_NUM'].replace('sin_registro',value='0')
temporal['PHONE_NUM']= temporal['PHONE_NUM'].replace('sin_registro',value='0')

In [35]:
temporal = temporal.replace("sin_registro", value=None)
doc_type = doc_type.replace("sin_registro", value=None)
student = student.replace({'DOC_TYPE': dic_doctype})
student = student.replace({'SCHOOL':dic_school})
student = student.replace("sin_registro", value='-')
school = school.replace("sin_registro", value=None)

In [36]:
temporal = temporal.astype({ 'CEL_NUM': 'int32','PHONE_NUM': 'int32'})
student = student.astype({'DOC_TYPE':'int32','SCHOOL':'int32'})

In [37]:
school.to_csv('C:/xampp/htdocs/EncuestaFCC/datos/SCHOOL.csv',header=False, encoding='utf-8',index=False)             
doc_type.to_csv('C:/xampp/htdocs/EncuestaFCC/datos/DOC_TYPE.csv',header=False, encoding='utf-8',index=False)
student.to_csv('C:/xampp/htdocs/EncuestaFCC/datos/STUDENT.csv',header=False, encoding='utf-8', index=False)
temporal.to_csv('C:/xampp/htdocs/EncuestaFCC/datos/TEMPORAL.csv',header=False, encoding='utf-8', index=False)

with xlwings.App(visible=True) as app:
    wb = xlwings.Book('EGRESA2.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('STUDENT.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('TEMPORAL.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()

In [45]:
student[student.DOC_NUM.duplicated() == True]

,COD,DOC_TYPE,DOC_NUM,PATERNAL_NAME,MATERNAL_NAME,NAME,SCHOOL
1307,17110516,1,77489392,CHAVEZ,CORNEJO,DERLY ANTONIO,1
2636,20110335,1,74178479,DIAZ,HUARI,STACY JAZMIN,2
3692,18110387,1,72084843,CHAPOÑAN,SANDOVAL,ROBERTSON ALBERTO,2
4175,16110152,1,70779113,YATACO,ZUTA,JESÚS MIGUEL ANGEL,2
6565,04110030,1,42281245,VILCA,AGUIRRE,FAUSTO JOSE,2
9296,992473,1,06805903,QUECCAÑO,CHOQQUE,CRISTINA,2
9361,997591,1,09917834,CARTOLIN,FERNANDEZ,OSCAR ALEXANDER,2
11004,12110264,1,47463049,AGUILAR,CUEVA,ERICK ERNESTO,3
11018,14110085,1,72098895,LÓPEZ,GUTIÉRREZ,CARLOS DANIEL,3
